In [39]:
import numpy as np
import math

In [38]:
# C(t) = e^(-r∆t) (q*C_u(t+∆t) + (1-q)*C_d(t+∆t))
# q = (e^(r∆t) - d)/(u - d)

def BinomialPricing_European(S,K,T,N,u,d,r):
  delta_t = T/N
  q = ((math.e)**(r*delta_t) - d)/(u-d)

  leaf_nodes = N + 1  # number of leaf nodes
  S_leaf_max  = S * (u ** N) # max stock price at leaf node the upper most one
  S_leaf_nodes = [(S_leaf_max * (d**i))/ (u ** i) for i in range(N+1)] # Stock @ maturity
  option_payoff = [max(0, S_leaf_nodes[i] - K) for i in range(len(S_leaf_nodes))] # Value of call @ maturity

  for i in range(N): # iterating back to 0th time period
    for j in range(N - i):
      C =  (math.e**(-r*delta_t)) * ((q*option_payoff[j]) + ((1-q)*option_payoff[j+1]))
      option_payoff[j] =  C

  return option_payoff[0]

In [43]:
S0 = 100      # Stock price @ t = 0
K = 100       # Strike price
T = 8         # Time to expiry (years)
r = 0.06      # risk-free rate (p.a)
N = 4        # steps
u = 2       # up factor
d = 1/u       # down factor (makes ud and du indifferent)

BinomialPricing_European(S0,K,T,N,u,d,r)

60.04530437255597

In [78]:
def BinomialPricing_American(S,K,T,N,u,d,r):
  delta_t = T/N
  q = ((math.e)**(r*delta_t) - d)/(u-d)

  stock = [] # contains stock prices from t = 0 - N + 1
  payoffs = []
  for i in range(N+1):
    stock.append( [S * (u ** j) * d ** (i - j) for j in range(i+1)][::-1] )
    payoffs.append( [0] * (i+1) )

  option_payoff = [max(0, K - stock[-1][i]) for i in range(len(stock[-1]))] # Value of put @ maturity
  payoffs[-1] = option_payoff

  for i in range(N-1,-1,-1): # iterating back to 0th time period
    for j in range(i+1):
      # print(i,j)
      value_of_keeping_option =  (math.e**(-r*delta_t)) * ((q*payoffs[i+1][j]) + ((1-q)*payoffs[i+1][j+1]))
      payoff_at_excercising  = max(0, K - stock[i][j])
      payoffs[i][j] = max(payoff_at_excercising,value_of_keeping_option)

  return payoffs[0][0]

In [77]:
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.08      # annual risk-free rate
N = 6         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree



print(BinomialPricing_American(S0,K,T,N,u,d,r))

6.3372891599536025
